In [ ]:
%pip install torchinfo

In [ ]:
import torch
from torch import nn
from torchinfo import summary
import torch.nn.functional as F

class InceptionBlock(nn.Module):
    def __init__(self, in_channels, c1, c2, c3, c4):
        super(InceptionBlock, self).__init__()
        self.p1 = nn.Conv2d(in_channels = in_channels, out_channels=c1, kernel_size=1)
        self.p2 = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=c2[0], kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=c2[0],out_channels=c2[1],kernel_size=3, padding=1)
        )
        self.p3 = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=c3[0],kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=c3[0],out_channels=c3[1],kernel_size=5, padding=2)
        )
        self.p4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, padding=1, stride=1),
            nn.Conv2d(in_channels=in_channels, out_channels=c4, kernel_size=1)
        )

    def forward(self,x):
        o1 = F.relu(self.p1(x))
        o2 = F.relu(self.p2(x))
        o3 = F.relu(self.p3(x))
        o4 = F.relu(self.p4(x))

        return torch.concat((o1,o2,o3,o4), dim=1)


class GoogLeNet(nn.Module):

    def __init__(self):
        super(GoogLeNet, self).__init__()

        self.b1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2,padding=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2,padding=1)
        )

        self.b2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=1, stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=192, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        self.inception1 = InceptionBlock(192,64,[96,128],[16,32],32)

        self.b3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.inception2 = InceptionBlock(256, 128, [128, 192], [32, 96], 64)
        self.inception3 = InceptionBlock(480, 192, [96,208],[16,48],64)
        self.inception4 = InceptionBlock(512, 160,[112,224],[24,64],64)
        self.inception5 = InceptionBlock(512, 128, [128, 256],[24,64],64)
        self.inception6 = InceptionBlock(512, 112, [128,288],[32,64],64)
        self.inception7 = InceptionBlock(528, 256, [160,320],[32,128],128)
        self.inception8 = InceptionBlock(832, 256, [160,320],[32,128],128)
        self.inception9 = InceptionBlock(832, 384,[192,384],[48,128],128)

        self.b4 = nn.Sequential(
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten(),
            nn.Linear(1024, 10)
        )
        for m in self.modules():
            if(isinstance(m, nn.Conv2d)):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)

    def forward(self, x):
        x = self.b1(x)
        x = self.b2(x)
        x = self.inception1(x)
        x = self.b3(x)
        x = self.inception2(x)
        x = self.inception3(x)
        x = self.inception4(x)
        x = self.inception5(x)
        x = self.inception6(x)
        x = self.inception7(x)
        x = self.b3(x)
        x = self.inception8(x)
        x = self.inception9(x)

        x = self.b4(x)

        return x

if __name__ == "__main__":
    if torch.backends.mps.is_available():
        device = torch.device("mps")
    elif torch.backends.mps.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")
    model = GoogLeNet().to(device)
    summary(model, (1,3,224,224))

In [ ]:
import torch
from torch import nn
from torchvision.datasets import FashionMNIST
from torchvision import transforms
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch.utils.data as Data

from torchinfo import summary
import copy
import time
from tqdm import tqdm
def train_val_data_process():
    train_data = FashionMNIST(root='./GoogLeNet/data',
                            train = True,
                            transform=transforms.Compose([
                            transforms.Resize(size=224), 
                            transforms.Grayscale(num_output_channels=3),
                            transforms.ToTensor()]),
                            download=True)
    
    train_data, val_data = Data.random_split(train_data, [round(0.8*len(train_data)), round(0.2*len(train_data))])

    train_dataloader = Data.DataLoader(dataset=train_data,
                                       batch_size = 64,
                                       shuffle=True,
                                       num_workers=0)
    val_dataloader = Data.DataLoader(dataset=val_data,
                                       batch_size = 64,
                                       shuffle=True,
                                       num_workers=0)
    
    return train_dataloader, val_dataloader

def train_model_process(model, train_dataloader, val_dataloader, epochs):
    if torch.backends.mps.is_available():
        device = torch.device("mps")
    elif torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")

    optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

    loss = nn.CrossEntropyLoss()
    
    model = model.to(device)

    best_model_wts = copy.deepcopy(model.state_dict())

    best_acc = 0.0
    
    train_loss_all = []
    val_loss_all = []
    train_acc_all = []
    val_acc_all = []


    for epoch in range(1,epochs+1):

        epoch_start_time = time.time()

        print(f'The {epoch}/{epochs} Epoch')
        print("-"*10)

        train_loss = 0
        train_acc = 0
        val_loss = 0
        val_acc = 0

        train_num = 0
        val_num = 0

        train_bar = tqdm(train_dataloader, desc="Train")
        for step, (b_x, b_y) in enumerate (train_bar):
            b_x = b_x.to(device)
            b_y = b_y.to(device)

            model.train()

            output = model(b_x)

            pre_lab = torch.argmax(output, dim = 1)

            data_loss = loss(output, b_y)

            optimizer.zero_grad()
            data_loss.backward()
            optimizer.step()

            train_loss+=data_loss.item()*b_x.size(0)
            train_acc += torch.sum(pre_lab == b_y.data)

            train_num += b_x.size(0)

        test_bar = tqdm(val_dataloader, desc="Val")
        for step, (b_x, b_y) in enumerate (test_bar):
            b_x = b_x.to(device)
            b_y = b_y.to(device)

            model.eval()
            output = model(b_x)

            pre_lab = torch.argmax(output, dim = 1)

            data_loss = loss(output, b_y)

            val_loss += data_loss.item()*b_x.size(0)
            val_acc += torch.sum(pre_lab == b_y.data)
            val_num += b_x.size(0)
        
        train_loss_all.append(train_loss/train_num)
        train_acc_all.append(train_acc.item()/train_num)

        val_loss_all.append(val_loss/val_num)
        val_acc_all.append(val_acc.item()/val_num)
        print(f"{epoch} Train Loss: {train_loss_all[-1]:.4f} Train Acc: {train_acc_all[-1]:.4f}")
        print(f"{epoch} Val Loss: {val_loss_all[-1]:.4f} Val Acc: {val_acc_all[-1]:.4f}")

        if val_acc_all[-1]>best_acc:
            best_acc = val_acc_all[-1]

            best_model_wts = copy.deepcopy(model.state_dict())
        time_use = time.time()-epoch_start_time
        print(f"Time cost in Epoch {epoch} : {time_use}s")



    
    torch.save(best_model_wts, 'GoogLeNet/best_model.pth')

    train_process = pd.DataFrame(data={"epoch":range(epochs),
                                        "train_loss_all":train_loss_all,
                                        "val_loss_all":val_loss_all,
                                        "train_acc_all":train_acc_all,
                                        "val_acc_all":val_acc_all})
    return train_process



def matplot_acc_loss(train_process):
    plt.figure(figsize=(12,4))
    plt.subplot(1, 2, 1)
    plt.plot(train_process["epoch"], train_process.train_loss_all, 'ro-', label = "train loss")
    plt.plot(train_process["epoch"], train_process.val_loss_all, 'bs-', label = "val loss")
    plt.legend()
    plt.xlabel("epoch")
    plt.ylabel("loss")


    plt.subplot(1, 2, 2)
    plt.plot(train_process["epoch"], train_process.train_acc_all, 'ro-', label = "train acc")
    plt.plot(train_process["epoch"], train_process.val_acc_all, 'bs-', label = "val acc")
    plt.legend()
    plt.xlabel("epoch")
    plt.ylabel("acc")
    plt.show()

if __name__  == "__main__":
    googLeNet = GoogLeNet()
    train_dataloader, val_dataloader = train_val_data_process()

    train_process = train_model_process(googLeNet, train_dataloader, val_dataloader, 20)

    matplot_acc_loss(train_process)


TypeError: GoogLeNet.forward() missing 1 required positional argument: 'x'

In [ ]:
import torch
from torch import nn
from torchvision.datasets import FashionMNIST
from torchvision import transforms
import matplotlib.pyplot as plt
import pandas as pd
import torch.utils.data as Data
from torchinfo import summary
import copy


def test_val_data_process():
    test_data = FashionMNIST(root='./GoogLeNet/data',
                            train = False,
                            transform=transforms.Compose([
                            transforms.Resize(size=224), 
                            transforms.Grayscale(num_output_channels=3),
                            transforms.ToTensor()]),
                            )
    

    test_dataloader = Data.DataLoader(dataset=test_data,
                                       batch_size = 32,
                                       shuffle=True,
                                       num_workers = 0)
    
    return test_dataloader

def test_model_process(model, test_dataloader):
    if torch.backends.mps.is_available():
        device = torch.device("mps")
    elif torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")
    
    model = model.to(device)
    test_acc = 0.0
    test_num = 0
    

    with torch.no_grad():
        for step, (b_x, b_y) in enumerate(test_dataloader):
            b_x = b_x.to(device)
            b_y = b_y.to(device)

            model.eval()
            output = model(b_x)
            pre_lab = torch.argmax(output, dim=1)

            test_acc += torch.sum(pre_lab==b_y.data)
            test_num += b_x.size(0)
        
    
    print(f"The accuracy of testing: {test_acc/test_num}")

if __name__ == "__main__":
    model = GoogLeNet()

    model.load_state_dict(torch.load('GoogLeNet/best_model.pth'))
    test_loader = test_val_data_process()

    test_model_process(model, test_loader)

TypeError: GoogLeNet.forward() missing 1 required positional argument: 'x'